# Breaking the Rules: When Traditional RCTs Aren't Possible

> *"Rules for thee, but not for me"* - When randomized controlled trials are impossible, impractical, or unethical, innovators find another way.

## The Revolutionary Mindset

Traditional statisticians might tell you that without randomization, you cannot establish causality. **They're wrong.** This tutorial will show you how to break free from the constraints of traditional experimental design and discover causal relationships in the wild.

### What You'll Learn

- Why randomization isn't always the answer
- Creative approaches to causal identification
- Real-world case studies of "impossible" causal inference
- How to think like a causal detective
- Methods that work when RCTs fail

### The Challenge

Imagine you're trying to answer these questions:
- Does a new city policy reduce crime? (Can't randomize cities)
- Do violent video games cause aggression? (Unethical to expose children)
- Does education improve lifetime earnings? (Can't deny education randomly)
- Does smoking cause cancer? (Unethical to assign smoking)

Traditional RCT advocates would say these questions are unanswerable with certainty. **Innovators prove them wrong.**

In [ ]:
# Setup: The Tools of Innovation
import os
import sys
import warnings

warnings.filterwarnings("ignore")

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../../"))
if project_root not in sys.path:
    sys.path.insert(0, os.path.join(project_root, "libs"))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Import our causal inference arsenal
from causal_inference.core.base import CovariateData, OutcomeData, TreatmentData
from causal_inference.estimators.difference_in_differences import (
    DifferenceInDifferencesEstimator,
)
from causal_inference.estimators.iv import IVEstimator
from causal_inference.estimators.regression_discontinuity import RDDEstimator

# Set the stage for innovation
np.random.seed(42)
plt.style.use("default")  # Use default style instead of deprecated seaborn style
sns.set_palette("husl")

print("🚀 Ready to break the rules and discover causal truth!")
print("💡 Remember: Innovation begins where conventional wisdom ends.")

## Method 1: Instrumental Variables - Finding Nature's Experiments

**The Big Idea**: When you can't randomize the treatment, find something else that randomly affects treatment assignment but only affects the outcome through treatment.

### Real-World Case Study: Education and Earnings

*Question*: Does education causally increase earnings?
*Problem*: We can't randomly assign education levels to people
*Innovation*: Use distance to college as an instrumental variable!

In [ ]:
# Case Study 1: Education and Earnings - The Impossible Made Possible


def generate_education_earnings_data(n=1000):
    """
    Generate synthetic data mimicking the education-earnings relationship
    with distance to college as an instrumental variable
    """
    # Background characteristics (confounders)
    family_income = np.random.normal(
        50000, 20000, n
    )  # Family income affects both education and earnings
    ability = np.random.normal(
        100, 15, n
    )  # Ability affects both education and earnings

    # Instrumental variable: Distance to nearest college (randomly distributed by geography)
    distance_to_college = np.random.exponential(20, n)  # Miles

    # First stage: Distance affects education (closer = more education)
    # Education also depends on family income and ability
    education_propensity = (
        12  # Base education
        + -0.05 * distance_to_college  # Distance reduces education
        + 0.00008 * family_income  # Family income increases education
        + 0.05 * ability  # Ability increases education
        + np.random.normal(0, 2, n)
    )  # Random variation

    years_education = np.clip(education_propensity, 8, 20)  # 8-20 years of education

    # Second stage: Education affects earnings
    # Earnings also depend on ability and family background
    true_education_effect = 3000  # True causal effect: $3000 per year of education

    annual_earnings = (
        20000  # Base earnings
        + true_education_effect * years_education  # True causal effect
        + 0.3 * family_income  # Family background effect
        + 800 * ability  # Ability effect
        + np.random.normal(0, 10000, n)
    )  # Random variation

    annual_earnings = np.clip(annual_earnings, 15000, 200000)  # Realistic range

    return pd.DataFrame(
        {
            "years_education": years_education,
            "annual_earnings": annual_earnings,
            "distance_to_college": distance_to_college,
            "family_income": family_income,
            "ability": ability,
        }
    ), true_education_effect


# Generate the data
education_data, true_effect = generate_education_earnings_data(1000)

print("📊 Education and Earnings Dataset Generated")
print(f"True causal effect of education: ${true_effect:,} per year")
print("\nData preview:")
print(education_data.head())

# Visualize the relationship
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Raw correlation (biased)
axes[0, 0].scatter(
    education_data["years_education"], education_data["annual_earnings"], alpha=0.6
)
correlation = np.corrcoef(
    education_data["years_education"], education_data["annual_earnings"]
)[0, 1]
axes[0, 0].set_title(
    f"Raw Correlation: Education vs Earnings\nr = {correlation:.3f} (BIASED!)"
)
axes[0, 0].set_xlabel("Years of Education")
axes[0, 0].set_ylabel("Annual Earnings ($)")

# First stage: Distance to college affects education
axes[0, 1].scatter(
    education_data["distance_to_college"], education_data["years_education"], alpha=0.6
)
first_stage_corr = np.corrcoef(
    education_data["distance_to_college"], education_data["years_education"]
)[0, 1]
axes[0, 1].set_title(
    f"First Stage: Distance vs Education\nr = {first_stage_corr:.3f} (Good!)"
)
axes[0, 1].set_xlabel("Distance to College (miles)")
axes[0, 1].set_ylabel("Years of Education")

# Reduced form: Distance to college affects earnings (only through education)
axes[1, 0].scatter(
    education_data["distance_to_college"], education_data["annual_earnings"], alpha=0.6
)
reduced_form_corr = np.corrcoef(
    education_data["distance_to_college"], education_data["annual_earnings"]
)[0, 1]
axes[1, 0].set_title(f"Reduced Form: Distance vs Earnings\nr = {reduced_form_corr:.3f}")
axes[1, 0].set_xlabel("Distance to College (miles)")
axes[1, 0].set_ylabel("Annual Earnings ($)")

# Distribution of the instrument
axes[1, 1].hist(education_data["distance_to_college"], bins=30, alpha=0.7)
axes[1, 1].set_title("Distribution of Instrumental Variable")
axes[1, 1].set_xlabel("Distance to College (miles)")
axes[1, 1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

print("\n🔍 What we see:")
print(
    f"• Raw correlation suggests ${correlation * 10000:.0f} per year of education (BIASED!)"
)
print(f"• First stage shows distance affects education: {first_stage_corr:.3f}")
print(f"• Reduced form shows distance affects earnings: {reduced_form_corr:.3f}")
print(f"• True effect: ${true_effect:,} per year")

In [ ]:
# Method 1: Instrumental Variables - Breaking the Randomization Rule

# Prepare data for IV estimation
treatment = TreatmentData(
    values=education_data["years_education"],
    name="years_education",
    treatment_type="continuous",
)

outcome = OutcomeData(
    values=education_data["annual_earnings"],
    name="annual_earnings",
    outcome_type="continuous",
)

# Instrumental variable: Distance to college
instrument = education_data[["distance_to_college"]]

# Control variables
covariates = CovariateData(
    values=education_data[["family_income", "ability"]],
    names=["family_income", "ability"],
)

print("🎯 Using Instrumental Variables to Identify Causal Effects")
print("📐 Instrument: Distance to college")
print("🎓 Treatment: Years of education")
print("💰 Outcome: Annual earnings")

# Check IV assumptions using available diagnostics
print("\n🔍 Checking IV Assumptions...")

# Manual check of first stage strength
from sklearn.linear_model import LinearRegression

# First stage regression
X_first_stage = education_data[["distance_to_college", "family_income", "ability"]]
y_first_stage = education_data["years_education"]
first_stage = LinearRegression().fit(X_first_stage, y_first_stage)

# Calculate F-statistic for instrument relevance
instrument_coef = first_stage.coef_[0]  # Coefficient on distance_to_college
y_pred = first_stage.predict(X_first_stage)
residuals = y_first_stage - y_pred
mse = np.mean(residuals**2)
se_instrument = np.sqrt(mse * np.linalg.inv(X_first_stage.T @ X_first_stage)[0, 0])
t_stat = instrument_coef / se_instrument
f_stat = t_stat**2

print(f"✅ Relevance (First stage F-stat > 10): {f_stat:.2f}")
if f_stat > 10:
    print("   🎯 Strong instrument - passes relevance test!")
else:
    print("   ⚠️ Weak instrument - may have bias issues")

print("⚠️  Exogeneity (untestable): Assume distance is random")
print("🎯 Exclusion (untestable): Distance only affects earnings through education")

# Estimate causal effect using IV
iv_estimator = IVEstimator(
    first_stage_model="linear", second_stage_model="linear", bootstrap_samples=100
)

iv_estimator.fit(treatment, outcome, covariates, instrument)
iv_effect = iv_estimator.estimate_ate()

print("\n🎉 BREAKTHROUGH RESULTS:")
print(f"💡 IV Estimate: ${iv_effect.ate:,.0f} per year of education")
print(f"🎯 True Effect: ${true_effect:,} per year")
print(
    f"📊 95% CI: [${iv_effect.confidence_interval[0]:,.0f}, ${iv_effect.confidence_interval[1]:,.0f}]"
)
print(f"✨ Bias from raw correlation: ${abs((correlation * 10000) - true_effect):,.0f}")

# Compare with naive approaches
naive_ols = np.polyfit(
    education_data["years_education"], education_data["annual_earnings"], 1
)[0]

print("\n🤔 Comparison of Methods:")
print(f"📈 Naive OLS (biased): ${naive_ols:,.0f}")
print(f"🎯 IV Estimate (unbiased): ${iv_effect.ate:,.0f}")
print(f"✅ True Effect: ${true_effect:,}")

bias_reduction = abs(naive_ols - true_effect) - abs(iv_effect.ate - true_effect)
print(f"🚀 Bias reduction: ${bias_reduction:,.0f}")

if abs(iv_effect.ate - true_effect) < abs(naive_ols - true_effect):
    print("\n🏆 VICTORY! IV estimation recovered the true causal effect!")
    print("🎯 This is why we break the rules - to find the truth!")
else:
    print("\n⚠️ Even IV has limitations, but it's still better than naive correlation!")

## Method 2: Regression Discontinuity - Exploiting Arbitrary Cutoffs

**The Innovation**: When treatment is assigned based on an arbitrary threshold, we can treat observations just above and below the cutoff as randomly assigned.

### Real-World Case Study: Financial Aid and College Success

*Question*: Does financial aid improve college graduation rates?
*Problem*: Students who get aid are different from those who don't
*Innovation*: Use the arbitrary GPA cutoff for aid eligibility!

In [ ]:
# Case Study 2: Financial Aid and College Success - The Cutoff Revolution


def generate_financial_aid_data(n=800):
    """
    Generate data for financial aid and college graduation
    with a sharp cutoff at GPA = 3.0 for aid eligibility
    """
    # Running variable: High school GPA (centered around cutoff)
    gpa_cutoff = 3.0
    high_school_gpa = np.random.normal(3.0, 0.5, n)
    high_school_gpa = np.clip(high_school_gpa, 1.5, 4.0)  # Realistic range

    # Sharp discontinuity: Aid assigned based on GPA >= 3.0
    receives_aid = (high_school_gpa >= gpa_cutoff).astype(int)

    # Background characteristics
    family_income = np.random.normal(40000, 15000, n)
    parent_college = np.random.binomial(1, 0.3, n)

    # Smooth relationship: GPA predicts success
    base_graduation_prob = 0.3 + 0.4 * (high_school_gpa - 1.5) / (4.0 - 1.5)

    # True causal effect of aid (the discontinuous jump)
    true_aid_effect = 0.25  # 25 percentage point increase

    # Final graduation probability
    graduation_prob = (
        base_graduation_prob
        + true_aid_effect * receives_aid
        + 0.1 * parent_college  # Parent education effect
        + np.random.normal(0, 0.1, n)
    )  # Random noise

    graduation_prob = np.clip(graduation_prob, 0.05, 0.95)
    graduated = np.random.binomial(1, graduation_prob, n)

    return pd.DataFrame(
        {
            "high_school_gpa": high_school_gpa,
            "receives_aid": receives_aid,
            "graduated": graduated,
            "family_income": family_income,
            "parent_college": parent_college,
            "distance_from_cutoff": high_school_gpa - gpa_cutoff,
        }
    ), true_aid_effect


# Generate the data
aid_data, true_aid_effect = generate_financial_aid_data(800)

print("🎓 Financial Aid and College Success Dataset Generated")
print(f"True causal effect of aid: {true_aid_effect:.1%} increase in graduation rate")
print("Aid cutoff: GPA ≥ 3.0")
print("\nData preview:")
print(aid_data.head())

# Visualize the discontinuity
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Treatment assignment (sharp discontinuity)
gpa_bins = np.linspace(
    aid_data["high_school_gpa"].min(), aid_data["high_school_gpa"].max(), 20
)
binned_data = (
    aid_data.groupby(pd.cut(aid_data["high_school_gpa"], gpa_bins))
    .agg({"receives_aid": "mean", "graduated": "mean", "high_school_gpa": "mean"})
    .dropna()
)

axes[0, 0].scatter(binned_data["high_school_gpa"], binned_data["receives_aid"], s=50)
axes[0, 0].axvline(x=3.0, color="red", linestyle="--", alpha=0.7, label="Aid Cutoff")
axes[0, 0].set_title("Treatment Assignment: Sharp Discontinuity at GPA = 3.0")
axes[0, 0].set_xlabel("High School GPA")
axes[0, 0].set_ylabel("Probability of Receiving Aid")
axes[0, 0].legend()
axes[0, 0].set_ylim(-0.1, 1.1)

# Outcome discontinuity (the causal effect!)
axes[0, 1].scatter(
    binned_data["high_school_gpa"], binned_data["graduated"], s=50, color="orange"
)
axes[0, 1].axvline(x=3.0, color="red", linestyle="--", alpha=0.7, label="Aid Cutoff")
axes[0, 1].set_title("Outcome: Graduation Rate Jumps at Cutoff!")
axes[0, 1].set_xlabel("High School GPA")
axes[0, 1].set_ylabel("Graduation Rate")
axes[0, 1].legend()

# Raw data scatter
colors = ["red" if aid == 0 else "blue" for aid in aid_data["receives_aid"]]
axes[1, 0].scatter(
    aid_data["high_school_gpa"], aid_data["graduated"], alpha=0.6, c=colors, s=20
)
axes[1, 0].axvline(x=3.0, color="black", linestyle="-", alpha=0.8, linewidth=2)
axes[1, 0].set_title("Raw Data: Red=No Aid, Blue=Aid")
axes[1, 0].set_xlabel("High School GPA")
axes[1, 0].set_ylabel("Graduated (0/1)")

# Histogram of running variable
axes[1, 1].hist(aid_data["high_school_gpa"], bins=30, alpha=0.7, edgecolor="black")
axes[1, 1].axvline(x=3.0, color="red", linestyle="--", alpha=0.7, label="Cutoff")
axes[1, 1].set_title("Distribution of Running Variable (GPA)")
axes[1, 1].set_xlabel("High School GPA")
axes[1, 1].set_ylabel("Frequency")
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Check if discontinuity is visible
left_of_cutoff = aid_data[aid_data["high_school_gpa"].between(2.9, 3.0)][
    "graduated"
].mean()
right_of_cutoff = aid_data[aid_data["high_school_gpa"].between(3.0, 3.1)][
    "graduated"
].mean()
observed_jump = right_of_cutoff - left_of_cutoff

print("\n🔍 Visual Evidence of Causal Effect:")
print(f"📉 Graduation rate just left of cutoff (2.9-3.0): {left_of_cutoff:.1%}")
print(f"📈 Graduation rate just right of cutoff (3.0-3.1): {right_of_cutoff:.1%}")
print(f"⚡ Observed jump at cutoff: {observed_jump:.1%}")
print(f"🎯 True causal effect: {true_aid_effect:.1%}")

In [ ]:
# Method 2: Regression Discontinuity Design - Exploiting Arbitrary Rules

# Prepare data for RDD estimation
treatment_rdd = TreatmentData(
    values=aid_data["receives_aid"], name="receives_aid", treatment_type="binary"
)

outcome_rdd = OutcomeData(
    values=aid_data["graduated"], name="graduated", outcome_type="binary"
)

# Running variable and cutoff
running_variable = aid_data["high_school_gpa"]
cutoff = 3.0

# Control variables (optional in RDD)
covariates_rdd = CovariateData(
    values=aid_data[["family_income", "parent_college"]],
    names=["family_income", "parent_college"],
)

print("📏 Using Regression Discontinuity Design")
print("🎯 Running variable: High school GPA")
print("✂️ Cutoff: 3.0 GPA")
print("🎓 Treatment: Financial aid")
print("📊 Outcome: College graduation")

# Estimate causal effect using RDD
rdd_estimator = RDDEstimator(
    cutoff=cutoff,
    kernel="triangular",
    bandwidth_method="optimal",
    bootstrap_samples=100,
)

print("\n🔧 Fitting RDD model...")
rdd_estimator.fit(treatment_rdd, outcome_rdd, running_variable, covariates_rdd)
rdd_effect = rdd_estimator.estimate_ate()

print("\n🎉 REGRESSION DISCONTINUITY RESULTS:")
print(f"💡 RDD Estimate: {rdd_effect.ate:.1%} increase in graduation rate")
print(f"🎯 True Effect: {true_aid_effect:.1%}")
print(
    f"📊 95% CI: [{rdd_effect.confidence_interval[0]:.1%}, {rdd_effect.confidence_interval[1]:.1%}]"
)
print(f"🎯 Effect is significant: {rdd_effect.is_significant}")

# Compare with naive comparison
naive_effect = (
    aid_data[aid_data["receives_aid"] == 1]["graduated"].mean()
    - aid_data[aid_data["receives_aid"] == 0]["graduated"].mean()
)

print("\n🤔 Comparison of Methods:")
print(f"📈 Naive comparison (biased): {naive_effect:.1%}")
print(f"📏 RDD estimate (unbiased): {rdd_effect.ate:.1%}")
print(f"✅ True effect: {true_aid_effect:.1%}")

if abs(rdd_effect.ate - true_aid_effect) < abs(naive_effect - true_aid_effect):
    print("\n🏆 ANOTHER VICTORY! RDD revealed the true causal effect!")
    print("🔥 By exploiting an arbitrary cutoff, we discovered causality!")
    print("💡 This is the power of thinking outside the RCT box!")

# Visualize the RDD results
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Plot the fitted discontinuity
gpa_range = np.linspace(
    aid_data["high_school_gpa"].min(), aid_data["high_school_gpa"].max(), 100
)
bandwidth = getattr(
    rdd_estimator, "bandwidth_", 0.5
)  # Use estimated or default bandwidth

# Create binned averages for visualization
n_bins = 20
gpa_bins = np.linspace(
    aid_data["high_school_gpa"].min(), aid_data["high_school_gpa"].max(), n_bins
)
binned_means = []
binned_gpa = []

for i in range(len(gpa_bins) - 1):
    mask = (aid_data["high_school_gpa"] >= gpa_bins[i]) & (
        aid_data["high_school_gpa"] < gpa_bins[i + 1]
    )
    if mask.sum() > 0:
        binned_means.append(aid_data[mask]["graduated"].mean())
        binned_gpa.append(aid_data[mask]["high_school_gpa"].mean())

# Plot binned data
ax.scatter(binned_gpa, binned_means, s=60, alpha=0.8, zorder=3)

# Highlight the discontinuity
ax.axvline(
    x=cutoff,
    color="red",
    linestyle="--",
    alpha=0.7,
    linewidth=2,
    label=f"Cutoff (GPA = {cutoff})",
)

# Add effect annotation
ax.annotate(
    f"Causal Effect: {rdd_effect.ate:.1%}",
    xy=(cutoff + 0.05, 0.5),
    fontsize=12,
    fontweight="bold",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
)

ax.set_xlabel("High School GPA (Running Variable)", fontsize=12)
ax.set_ylabel("Graduation Rate", fontsize=12)
ax.set_title(
    "Regression Discontinuity: Financial Aid Effect on Graduation",
    fontsize=14,
    fontweight="bold",
)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 Key Insight: Students just above vs just below the GPA cutoff")
print("   are essentially identical except for aid - this is nature's experiment!")

## Method 3: Difference-in-Differences - Using Time as Your Friend

**The Revolution**: When treatment happens to some groups but not others at a specific time, we can use the before-after, treatment-control comparison to identify causal effects.

### Real-World Case Study: Minimum Wage and Employment

*Question*: Does raising minimum wage reduce employment?
*Problem*: States that raise wages are different from those that don't
*Innovation*: Compare changes in employment before and after wage increases!

In [ ]:
# Case Study 3: Minimum Wage Policy - Time Reveals Truth


def generate_minimum_wage_data():
    """
    Generate synthetic minimum wage and employment data
    Treatment group (New Jersey) raises wage in 1994
    Control group (Pennsylvania) does not
    """
    np.random.seed(42)  # For reproducibility

    # Time periods
    years = [1993, 1994, 1995, 1996]  # Pre and post treatment
    states = ["New Jersey (Treatment)", "Pennsylvania (Control)"]

    data = []

    # True effect: Minimum wage increase has NO effect on employment (contrary to theory!)
    true_effect = 0.0  # Revolutionary finding!

    for state in states:
        is_treatment = "Treatment" in state

        for year in years:
            # Treatment occurs in 1994 for New Jersey
            post_treatment = year >= 1994
            treated = is_treatment and post_treatment

            # Base employment rate with state-specific differences
            if is_treatment:
                base_employment = 0.72  # NJ baseline slightly higher
            else:
                base_employment = 0.70  # PA baseline

            # Common time trend (economic recovery)
            time_trend = 0.01 * (year - 1993)  # 1% per year improvement

            # Treatment effect (the key parameter!)
            treatment_effect = true_effect if treated else 0.0

            # Final employment rate
            employment_rate = (
                base_employment
                + time_trend
                + treatment_effect
                + np.random.normal(0, 0.02)
            )  # Noise

            employment_rate = np.clip(employment_rate, 0.5, 0.95)

            # Minimum wage
            if is_treatment and post_treatment:
                min_wage = 5.05  # Raised wage
            elif is_treatment:
                min_wage = 4.25  # Pre-treatment
            else:
                min_wage = 4.25  # Control always at federal minimum

            data.append(
                {
                    "state": state,
                    "year": year,
                    "is_treatment_state": is_treatment,
                    "post_treatment": post_treatment,
                    "treated": treated,
                    "employment_rate": employment_rate,
                    "min_wage": min_wage,
                }
            )

    return pd.DataFrame(data), true_effect


# Generate the data
wage_data, true_wage_effect = generate_minimum_wage_data()

print("💼 Minimum Wage and Employment Dataset Generated")
print(
    f"True effect of minimum wage increase: {true_wage_effect:.1%} change in employment"
)
print("Treatment: New Jersey raises minimum wage in 1994")
print("Control: Pennsylvania keeps wage constant")
print("\nData preview:")
print(wage_data)

# Visualize the difference-in-differences setup
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Employment trends
for state in wage_data["state"].unique():
    state_data = wage_data[wage_data["state"] == state]
    color = "blue" if "Treatment" in state else "red"
    linestyle = "-" if "Treatment" in state else "--"
    axes[0].plot(
        state_data["year"],
        state_data["employment_rate"],
        marker="o",
        color=color,
        linestyle=linestyle,
        linewidth=2,
        markersize=8,
        label=state,
    )

# Mark treatment period
axes[0].axvline(
    x=1993.5, color="gray", linestyle=":", alpha=0.7, label="Treatment Starts"
)
axes[0].set_title(
    "Employment Trends: Treatment vs Control", fontsize=12, fontweight="bold"
)
axes[0].set_xlabel("Year")
axes[0].set_ylabel("Employment Rate")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Minimum wage levels
for state in wage_data["state"].unique():
    state_data = wage_data[wage_data["state"] == state]
    color = "blue" if "Treatment" in state else "red"
    linestyle = "-" if "Treatment" in state else "--"
    axes[1].plot(
        state_data["year"],
        state_data["min_wage"],
        marker="s",
        color=color,
        linestyle=linestyle,
        linewidth=2,
        markersize=8,
        label=state,
    )

axes[1].axvline(
    x=1993.5, color="gray", linestyle=":", alpha=0.7, label="Treatment Starts"
)
axes[1].set_title("Minimum Wage Levels", fontsize=12, fontweight="bold")
axes[1].set_xlabel("Year")
axes[1].set_ylabel("Minimum Wage ($)")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate components of DiD
pre_treatment = wage_data[wage_data["post_treatment"] == False]
post_treatment = wage_data[wage_data["post_treatment"] == True]

# Before-after difference for treatment group
nj_pre = pre_treatment[pre_treatment["is_treatment_state"]]["employment_rate"].mean()
nj_post = post_treatment[post_treatment["is_treatment_state"]]["employment_rate"].mean()
nj_change = nj_post - nj_pre

# Before-after difference for control group
pa_pre = pre_treatment[~pre_treatment["is_treatment_state"]]["employment_rate"].mean()
pa_post = post_treatment[~post_treatment["is_treatment_state"]][
    "employment_rate"
].mean()
pa_change = pa_post - pa_pre

# Difference-in-differences
did_effect = nj_change - pa_change

print("\n📊 Difference-in-Differences Components:")
print(f"🔵 NJ (Treatment): {nj_pre:.1%} → {nj_post:.1%} (Change: {nj_change:+.1%})")
print(f"🔴 PA (Control): {pa_pre:.1%} → {pa_post:.1%} (Change: {pa_change:+.1%})")
print(f"⚡ Difference-in-Differences: {did_effect:+.1%}")
print(f"🎯 True effect: {true_wage_effect:.1%}")

In [ ]:
# Method 3: Difference-in-Differences - Time Reveals Causality

# Prepare data for DiD estimation
treatment_did = TreatmentData(
    values=wage_data["treated"].astype(int), name="treated", treatment_type="binary"
)

outcome_did = OutcomeData(
    values=wage_data["employment_rate"],
    name="employment_rate",
    outcome_type="continuous",
)

# Group and time indicators
group_indicator = wage_data["is_treatment_state"].astype(int)
time_indicator = wage_data["post_treatment"].astype(int)

print("⏰ Using Difference-in-Differences Design")
print("🏛️ Treatment Group: New Jersey (raised minimum wage)")
print("🏛️ Control Group: Pennsylvania (kept wage constant)")
print("📅 Treatment Time: 1994 onwards")
print("💼 Outcome: Employment rate")

# Estimate causal effect using DiD
did_estimator = DifferenceInDifferencesEstimator(bootstrap_samples=100)

print("\n🔧 Fitting DiD model...")
did_estimator.fit(
    treatment=treatment_did,
    outcome=outcome_did,
    group_indicator=group_indicator,
    time_indicator=time_indicator,
)

did_effect_result = did_estimator.estimate_ate()

print("\n🎉 DIFFERENCE-IN-DIFFERENCES RESULTS:")
print(f"💡 DiD Estimate: {did_effect_result.ate:+.1%} change in employment")
print(f"🎯 True Effect: {true_wage_effect:.1%}")
print(
    f"📊 95% CI: [{did_effect_result.confidence_interval[0]:+.1%}, {did_effect_result.confidence_interval[1]:+.1%}]"
)
print(f"🎯 Effect is significant: {did_effect_result.is_significant}")

# Manual calculation for verification
print(f"\n🧮 Manual DiD Calculation: {did_effect:+.1%} (matches!)")

# Compare with naive approaches
naive_before_after = nj_change
naive_cross_section = (
    post_treatment[post_treatment["is_treatment_state"]]["employment_rate"].mean()
    - post_treatment[~post_treatment["is_treatment_state"]]["employment_rate"].mean()
)

print("\n🤔 Comparison of Methods:")
print(f"📈 Naive before-after (biased): {naive_before_after:+.1%}")
print(f"📊 Naive cross-section (biased): {naive_cross_section:+.1%}")
print(f"⏰ DiD estimate (unbiased): {did_effect_result.ate:+.1%}")
print(f"✅ True effect: {true_wage_effect:.1%}")

print("\n🧠 Key Insights:")
print(f"• Before-after comparison is biased by time trends ({pa_change:+.1%})")
print("• Cross-section comparison is biased by state differences")
print("• DiD removes BOTH biases by using the control group!")

if abs(did_effect_result.ate - true_wage_effect) < 0.02:  # Within 2 percentage points
    print("\n🏆 REVOLUTIONARY SUCCESS! DiD revealed the truth!")
    print("🔥 Minimum wage increases don't hurt employment!")
    print("💡 This challenges conventional economic theory!")
    print("🌟 By using time cleverly, we discovered causality!")

# Create DiD visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Calculate group means by period
means = (
    wage_data.groupby(["is_treatment_state", "post_treatment"])["employment_rate"]
    .mean()
    .unstack()
)

# Plot the DiD
periods = ["Pre-Treatment", "Post-Treatment"]
treatment_means = [means.loc[True, False], means.loc[True, True]]
control_means = [means.loc[False, False], means.loc[False, True]]

ax.plot(
    periods,
    treatment_means,
    "o-",
    color="blue",
    linewidth=3,
    markersize=10,
    label="Treatment (New Jersey)",
)
ax.plot(
    periods,
    control_means,
    "o--",
    color="red",
    linewidth=3,
    markersize=10,
    label="Control (Pennsylvania)",
)

# Add effect annotation
mid_treatment = np.mean(treatment_means)
ax.annotate(
    f"DiD Effect: {did_effect_result.ate:+.1%}",
    xy=(1, treatment_means[1]),
    xytext=(1.2, mid_treatment),
    fontsize=12,
    fontweight="bold",
    arrowprops=dict(arrowstyle="->", color="black", lw=1.5),
    bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.8),
)

ax.set_ylabel("Employment Rate", fontsize=12)
ax.set_title(
    "Difference-in-Differences: Minimum Wage Effect", fontsize=14, fontweight="bold"
)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 The Power of Innovation: By combining space AND time,")
print("   DiD eliminated confounding and revealed the true causal effect!")

## The Revolutionary Mindset: Key Principles

### 1. Question Everything
- **Traditional thinking**: "We need randomization for causality"
- **Revolutionary thinking**: "Nature provides experiments everywhere"

### 2. Look for Natural Experiments
- Geographic boundaries (states, countries)
- Arbitrary cutoffs (GPA thresholds, age limits)
- Policy changes (laws, regulations)
- Random events (lotteries, weather)

### 3. Exploit Institutional Rules
- School assignment boundaries
- Eligibility thresholds
- Bureaucratic procedures
- Historical accidents

### 4. Think Like a Detective
- What creates variation in treatment?
- Is this variation "as good as random"?
- What other factors could explain the outcome?
- How can I rule out alternative explanations?

## Challenge Exercise: Find Your Own Natural Experiment

Think of a causal question in your field where randomization is impossible. Then:

1. **Identify potential instruments** - What randomly affects your treatment?
2. **Look for discontinuities** - Are there arbitrary cutoffs in your domain?
3. **Find natural experiments** - What events create random variation?
4. **Design your approach** - How would you exploit this variation?

### Examples to Inspire You:

- **Marketing**: Store opening decisions based on population thresholds
- **Healthcare**: Hospital admission cutoffs based on bed availability  
- **Education**: Classroom size limits creating random assignment
- **Technology**: Server assignments based on user ID ranges
- **Finance**: Credit decisions based on arbitrary score cutoffs

In [ ]:
# Your Challenge: Design a Natural Experiment

print("🎯 CHALLENGE: Design Your Own Natural Experiment")
print("=" * 50)
print()
print("Step 1: Choose a causal question where RCTs are impossible")
print("Examples:")
print("• Does city size affect innovation rates?")
print("• Do strict parents create more successful children?")
print("• Does social media use cause depression?")
print("• Do open offices increase productivity?")
print()
print("Step 2: Identify your identification strategy")
print("Options:")
print("🎻 Instrumental Variables: Find something that randomly affects treatment")
print("📏 Regression Discontinuity: Look for arbitrary cutoffs")
print("⏰ Difference-in-Differences: Find policy changes or events")
print()
print("Step 3: Evaluate your approach")
print("Questions to ask:")
print("• Is the variation truly random (or as good as random)?")
print("• Can I defend the identifying assumptions?")
print("• What are the potential threats to validity?")
print("• How would I test my assumptions?")

# Example template for your own analysis
my_research_question = """
YOUR RESEARCH QUESTION HERE:
Example: Does working from home increase productivity?

WHY RCTs ARE IMPOSSIBLE:
Example: Companies can't randomly assign remote work policies

MY IDENTIFICATION STRATEGY:
Example: Use snow storms as an instrument - they force remote work randomly

ASSUMPTIONS I NEED TO DEFEND:
Example: Snow storms only affect productivity through remote work

HOW I WOULD TEST THIS:
Example: Check if snow affects productivity in jobs that can't be done remotely
"""

print("\n📝 Template for Your Analysis:")
print(my_research_question)

print("\n🌟 Remember: Every 'impossible' causal question")
print("   is an opportunity for innovation!")
print("\n💡 The best causal inference comes from creative thinking,")
print("   not just following textbook methods!")

## Summary: Breaking Free from Traditional Constraints

### What We've Learned

1. **Randomization is not the only path to causality**
   - Nature provides experiments everywhere
   - Institutional rules create random variation
   - Historical events offer natural experiments

2. **Three powerful alternatives to RCTs**:
   - **Instrumental Variables**: Exploit random factors that affect treatment
   - **Regression Discontinuity**: Use arbitrary cutoffs as natural experiments
   - **Difference-in-Differences**: Combine time and group variation

3. **The innovative mindset**:
   - Question conventional wisdom
   - Look for creative solutions
   - Exploit institutional features
   - Think like a detective

### Key Takeaways

- 🚫 **Don't accept "we can't randomize" as the end of the conversation**
- 🔍 **Look for sources of random or quasi-random variation**
- 🏗️ **Exploit institutional rules and arbitrary cutoffs**
- ⏰ **Use time and policy changes to your advantage**
- 🧠 **Creative thinking is your most powerful tool**

### Next Steps

Continue your journey of innovation with:
- **Tutorial 2**: Seeing Different - Identifying hidden causal relationships
- **Tutorial 3**: The Crazy Idea - Using ML for causal inference
- **Tutorial 4**: Change Things - From analysis to intervention

---

*"The people who are crazy enough to think they can change the world are the ones who do." - Keep breaking rules and discovering truth!*